In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import jieba
from sklearnex import patch_sklearn

In [2]:
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
train_data=pd.read_csv("/data/linqinhong/kaggle/yitushibie/data/yitushibie/train.csv",header=None,names=["text","title"],sep="\t")
test_data=pd.read_csv("/data/linqinhong/kaggle/yitushibie/data/yitushibie/test.csv",header=None,names=["text","title"],sep="\t")

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
from nltk.corpus import stopwords
stopword=stopwords.words("chinese")

In [6]:
tfvec=TfidfVectorizer(tokenizer=jieba.lcut,stop_words=stopword)

In [7]:
tfvec_fit=tfvec.fit_transform(list(train_data["text"]))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.604 seconds.
Prefix dict has been built successfully.
/opt/anaconda3/envs/tweet/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['下', '不', '为什', '什', '今', '使', '先', '却', '只', '唷', '啪', '喔', '天', '好', '後', '最', '漫', '然', '特', '特别', '见', '设', '说', '达', '面', '麽'] not in stop_words.
  % sorted(inconsistent)


In [8]:
tfvec_result=tfvec_fit.toarray()

In [9]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [10]:
classifier=SVC(kernel="linear",random_state=3407,C=1.0)
classifierKNN=KNeighborsClassifier()

In [11]:
labelsset=set(train_data["title"])
labelsdict={}
reverser_dict={}
for index,keys in enumerate(labelsset):
    labelsdict[keys]=index
for index,keys in enumerate(labelsset):
    reverser_dict[index]=keys

In [12]:
def convert(text):
    return labelsdict[text]

In [13]:
train_data["label"]=train_data["title"].apply(convert)

In [14]:
from sklearn.model_selection import KFold
KF=KFold(n_splits=5,shuffle=True)


In [15]:
result=KF.split(train_data,train_data["label"])

In [16]:
import torch
device=torch.device("cuda:2") if torch.cuda.is_available()==True else torch.device("cpu")

In [17]:
score=[]
for x_split,y_split in result:
    x1=tfvec_result[list(x_split)]
    y1=np.array(train_data["label"][x_split])
    x2=tfvec_result[list(y_split)]
    y2=np.array(train_data["label"][y_split])
    classifierKNN.fit(X=x1,y=y1)
    a=classifierKNN.score(x2,y2)
    score.append(a)

KeyboardInterrupt: 

In [20]:
x1.shape

(9680, 10720)

In [ ]:
classifierKNN.get_params()

In [ ]:
from sklearn.metrics import classification_report
print(score)

In [ ]:
import pickle

In [ ]:
"""with open("/data/linqinhong/kaggle/yitushibie/data/yitushibie/model/KNN.pickle","wb") as f:
    pickle.dump(classifierKNN,f)"""

In [ ]:
test_trans=tfvec.transform(list(test_data["text"]))

In [ ]:
test_result=test_trans.toarray()

In [ ]:
test_result.shape

In [ ]:
pred=classifierKNN.predict(test_result[list(range(len(test_result)))])

In [ ]:
pred_text=[reverser_dict[i] for i in pred]

In [ ]:
test_data["title"]=pred_text

In [ ]:
test_data

In [ ]:
test_data.to_csv("/data/linqinhong/kaggle/yitushibie/data/yitushibie/resultKNN.csv")